In [ ]:
# Testing the fine tuned model
import sys
import os
# Add the project root directory to the PYTHONPATH
project_root = os.path.abspath(os.path.join(".."))
print(f"Project root: {project_root}")
sys.path.append(project_root)
from agents.generator import Generator

model_lora = Generator(
    base_model_name="mistralai/Mistral-7B-v0.1", 
    adapter_dir="../saved_models/lora_finetuned/checkpoint-72000", 
    device="cpu"
)

base_model = Generator(
    base_model_name="mistralai/Mistral-7B-v0.1", 
    device="cpu",
    finetuned=False
)

In [ ]:
# prompt = "QUESTION: What are the symptoms of Alzheimer's disease?\nCONTEXT:\nAlzheimer's disease is a progressive neurological disorder.\nEarly symptoms include memory loss and confusion."
prompt = "QUESTION: What are the symptoms of Alzheimer's disease?"
response = model_lora.generate(prompt, temperature=0.7, max_length=200, top_p=0.95, top_k=50)
print(response)

print("=============================================")

response_base = base_model.generate(prompt, temperature=0.7, max_length=200, top_p=0.95, top_k=50)
print(response_base)

In [ ]:
# Testing the RAG pipeline
from scripts.rag_pipeline import RAGPipeline
pipeline = RAGPipeline(
        retriever_path="../data/rag/rag_data.txt",
        context_model_name="mistralai/Mistral-7B-v0.1",
        generator_model_name="mistralai/Mistral-7B-v0.1",
        generator_model_dir="../saved_models/lora_finetuned/checkpoint-72000",
        fact_checker_model="all-mpnet-base-v2",
        summarizer_model="facebook/bart-large-cnn",
        device="cpu"
)   
query = "What are the symptoms of Alzheimer's disease?"
final_response = pipeline.run(
        query, 
        top_k_ret=3,
        top_k_gen=100,
        top_p_gen=0.95,
        max_length_gen=512
)
print("\nFinal Response:", final_response)